In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score

# Define transformations
img_height, img_width = 224, 224
train_transforms = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transforms = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

NUM_WORKERS = os.cpu_count()

def create_dataloaders(train_dir, test_dir, train_transform, test_transform, batch_size, num_workers=NUM_WORKERS):
    train_data = datasets.ImageFolder(train_dir, transform=train_transform)
    test_data = datasets.ImageFolder(test_dir, transform=test_transform)
    class_names = train_data.classes

    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    return train_dataloader, test_dataloader, class_names

# Setup dataloaders
# Setup directory paths to train and test images
train_dir=r'F:\ML_Projects\Image_Classification\niharika\DATASET\TRAIN'
test_dir =  r'F:\ML_Projects\Image_Classification\niharika\DATASET\TEST'

train_dataloader, test_dataloader, class_names = create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    train_transform=train_transforms,
    test_transform=test_transforms,
    batch_size=32
)

model = models.vgg19(pretrained=True)
# Replace the final layer of the classifier to match the number of classes
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, len(class_names))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = 0
    total = 0
    predicted_labels = []
    true_labels = []
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predicted_labels.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = correct / total
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    f1 = f1_score(true_labels, predicted_labels, average='weighted')

    print(f"Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")


E:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
E:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
E:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [1/20], Accuracy: 0.3, Precision: 0.1744248778711947, Recall: 0.3, F1 Score: 0.20998576241197656


E:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [2/20], Accuracy: 0.4021276595744681, Precision: 0.44829757145023785, Recall: 0.4021276595744681, F1 Score: 0.27715435103438435
Epoch [3/20], Accuracy: 0.5723404255319149, Precision: 0.5881331940629863, Recall: 0.5723404255319149, F1 Score: 0.5502568724807451
Epoch [4/20], Accuracy: 0.7170212765957447, Precision: 0.7641284091368126, Recall: 0.7170212765957447, F1 Score: 0.7147820081441125
Epoch [5/20], Accuracy: 0.7893617021276595, Precision: 0.786713564473752, Recall: 0.7893617021276595, F1 Score: 0.7876556509902669
Epoch [6/20], Accuracy: 0.8468085106382979, Precision: 0.854245593215472, Recall: 0.8468085106382979, F1 Score: 0.8449478944652675
Epoch [7/20], Accuracy: 0.7893617021276595, Precision: 0.8180337614458625, Recall: 0.7893617021276595, F1 Score: 0.7827071291745064
Epoch [8/20], Accuracy: 0.8404255319148937, Precision: 0.8495300161521869, Recall: 0.8404255319148937, F1 Score: 0.8310810727206642
Epoch [9/20], Accuracy: 0.8851063829787233, Precision: 0.890097419563602, Re

In [2]:
import torch

# Assuming `model` is your PyTorch model
torch.save(model.state_dict(), 'model3_inception_n.pth')

In [6]:
import matplotlib.pyplot as plt

def plot_loss_curves(results):
    """Plot training and testing loss curves."""
    train_loss = results["train_loss"]
    test_loss = results["test_loss"]

    epochs = range(1, len(train_loss) + 1)

    plt.plot(epochs, train_loss, label="Training Loss")
    plt.plot(epochs, test_loss, label="Testing Loss")
    plt.title("Training and Testing Loss Curves")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

# Example usage:
plot_loss_curves(VGG)

NameError: name 'VGG' is not defined